In [17]:
# ass 4 matrix multiplication
%%writefile ass4matrixmultiplication.cu


// matrix_multiplication.cu

#include <iostream>
using namespace std;

__global__ void multiply(int* A, int* B, int* C, int size) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < size && col < size) {
        int sum = 0;
        for (int i = 0; i < size; i++) {
            sum += A[row * size + i] * B[i * size + col];
        }
        C[row * size + col] = sum;
    }
}

int main() {
    const int N = 2;
    const int matrixSize = N * N;
    const size_t matrixBytes = matrixSize * sizeof(int);

    int* A, * B, * C;
    A = new int[matrixSize];
    B = new int[matrixSize];
    C = new int[matrixSize];

    // Initialize matrices A and B
    for (int i = 0; i < matrixSize; i++) {
        A[i] = rand() % 10;
        B[i] = rand() % 10;
    }

    // Print matrices A and B
    cout << "Matrix A: \n";
    for (int i = 0; i < matrixSize; i++) {
        cout << A[i] << " ";
        if ((i + 1) % N == 0) cout << endl;
    }
    cout << endl;

    cout << "Matrix B: \n";
    for (int i = 0; i < matrixSize; i++) {
        cout << B[i] << " ";
        if ((i + 1) % N == 0) cout << endl;
    }
    cout << endl;

    int* d_A, * d_B, * d_C;
    cudaMalloc(&d_A, matrixBytes);
    cudaMalloc(&d_B, matrixBytes);
    cudaMalloc(&d_C, matrixBytes);

    cudaMemcpy(d_A, A, matrixBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, matrixBytes, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((N + 15) / 16, (N + 15) / 16);

    multiply<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, N);

    cudaMemcpy(C, d_C, matrixBytes, cudaMemcpyDeviceToHost);

    // Print result matrix C
    cout << "Result Matrix C: \n";
    for (int i = 0; i < matrixSize; i++) {
        cout << C[i] << " ";
        if ((i + 1) % N == 0) cout << endl;
    }
    cout << endl;

    delete[] A;
    delete[] B;
    delete[] C;
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Writing ass4matrixmultiplication.cu


In [18]:
!nvcc -o ass4matrixmultiplication ass4matrixmultiplication.cu -Xcompiler -fopenmp
!./ass4matrixmultiplication

Matrix A: 
3 7 
3 6 

Matrix B: 
6 5 
5 2 

Result Matrix C: 
0 0 
0 0 

